In [101]:
import numpy as np
import keras
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from functools import wraps
from scipy.sparse import csr_matrix
from tensorflow import keras
import tensorflow as tf
import re
from tensorflow.keras import layers
from typing import Any, Tuple, Dict, Union,Callable
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.preprocessing import LabelEncoder

In [87]:
pd.options.display.float_format = '{:.0f}'.format

In [88]:
dados = pd.read_csv(r"C:\estudoia\excelfiles\data_superstore.csv",sep=";")
dados.fillna(0, inplace=True)

In [89]:
dados = dados.apply(
    lambda col: pd.to_datetime(col, errors='coerce') if col.name in ['OrderDate', 'ShipDate']
    else (col.apply(lambda k: float(k.replace(",", ".")) if isinstance(
        k, str) and col.name in ['Sales', 'Profit'] else k) if col.name in ['Sales', 'Profit', 'Discount']
          else (col.astype(int) if col.name == 'Quantity' else col))
)


In [90]:


dados[["ProductID", "OrderID"]] = dados[["ProductID", "OrderID"]].apply(
    lambda col: col.apply(lambda k: re.sub(r'.*-(\d+)', r'\1', str(k)) if col.name == "ProductID" else re.sub(
        r'[-CAUS]+', '', str(k))), axis=0)

In [91]:
def generate_id_decorator(*columns: Tuple) -> Any:
    def decorator(func):
        def wrapper(df, *args, **kwargs):
            for column_name in columns:
                unique_values = df[column_name].unique()
                id_mapping = {value: idx for idx, value in enumerate(unique_values)}
                new_column_name = f'{column_name}ID'
                df[new_column_name] = df[column_name].map(id_mapping)
            return func(df, *args, **kwargs)
        return wrapper
    return decorator

In [92]:
dados = dados.apply(lambda col: pd.to_datetime(col, errors='coerce') if col.name in ['OrderDate', 'ShipDate']
                    else col.apply(lambda k: float(k.replace(",", ".")) if isinstance(k, str) else k) if col.name in ['Sales', 'Profit', 'Discount']
                    else col.astype(int) if col.name == 'Quantity'
                    else col)


In [93]:



dados = dados.groupby(['CustomerID', 'CustomerName', 'Segment', 'Country', 'City', 'State', 'PostalCode',
                                 'Region', 'ProductID', 'Category', 'Sub-Category', 'ProductName']).agg({
                                     'Sales': 'sum',
                                     'Quantity': 'sum',
                                     'Discount': 'sum'
                                 }).reset_index()


print(dados)


     CustomerID        CustomerName   Segment        Country           City  \
0      AA-10315          Alex Avila  Consumer  United States    Minneapolis   
1      AA-10315          Alex Avila  Consumer  United States    Minneapolis   
2      AA-10315          Alex Avila  Consumer  United States  New York City   
3      AA-10315          Alex Avila  Consumer  United States  New York City   
4      AA-10315          Alex Avila  Consumer  United States     Round Rock   
...         ...                 ...       ...            ...            ...   
9981   ZD-21925  Zuschuss Donatelli  Consumer  United States       Richmond   
9982   ZD-21925  Zuschuss Donatelli  Consumer  United States       Richmond   
9983   ZD-21925  Zuschuss Donatelli  Consumer  United States  San Francisco   
9984   ZD-21925  Zuschuss Donatelli  Consumer  United States  San Francisco   
9985   ZD-21925  Zuschuss Donatelli  Consumer  United States  San Francisco   

           State  PostalCode   Region ProductID    

In [94]:
dados

,CustomerID,CustomerName,Segment,Country,City,State,PostalCode,Region,ProductID,Category,Sub-Category,ProductName,Sales,Quantity,Discount
0,AA-10315,Alex Avila,Consumer,United States,Minneapolis,Minnesota,55407,Central,10000576,Office Supplies,Appliances,"Belkin 325VA UPS Surge Protector, 6'",363,3,0
1,AA-10315,Alex Avila,Consumer,United States,Minneapolis,Minnesota,55407,Central,10004654,Office Supplies,Binders,Avery Binding System Hidden Tab Executive Styl...,12,2,0
2,AA-10315,Alex Avila,Consumer,United States,New York City,New York,10011,East,10000349,Office Supplies,Paper,Easy-staple paper,15,3,0
3,AA-10315,Alex Avila,Consumer,United States,New York City,New York,10011,East,10002456,Furniture,Furnishings,"Master Caster Door Stop, Large Neon Orange",15,2,0
4,AA-10315,Alex Avila,Consumer,United States,Round Rock,Texas,78664,Central,10000151,Office Supplies,Supplies,High Speed Automatic Electric Letter Opener,3930,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,ZD-21925,Zuschuss Donatelli,Consumer,United States,Richmond,Indiana,47374,Central,10000673,Technology,Phones,Plantronics Voyager Pro HD - Bluetooth Headset,260,4,0
9982,ZD-21925,Zuschuss Donatelli,Consumer,United States,Richmond,Indiana,47374,Central,10003799,Furniture,Furnishings,"Seth Thomas 13 1/2"" Wall Clock",71,4,0
9983,ZD-21925,Zuschuss Donatelli,Consumer,United States,San Francisco,California,94109,West,10001949,Technology,Phones,Cisco SPA 501G IP Phone,213,3,0
9984,ZD-21925,Zuschuss Donatelli,Consumer,United States,San Francisco,California,94109,West,10002215,Office Supplies,Binders,"Wilson Jones Hanging View Binder, White, 1""",23,4,0


In [95]:

def one_hot_encode_all_categorical(func: Any, test_size: float = 0.33, random_state: int = 42) -> pd.DataFrame:
    @wraps(func)
    def wrapper(*args: Tuple, **kwargs: Dict[str, Any]) -> Union[pd.DataFrame, csr_matrix]:
        result = func(*args, **kwargs)
        
        if not isinstance(result, pd.DataFrame):
            raise ValueError("Not Found")
        
        data_cols = ['OrderDate', 'ShipDate']
        if all(col in result.columns for col in data_cols):
            result = result.drop(columns=data_cols)
        
        categorical_cols = result.select_dtypes(include=['object']).columns
        
        if categorical_cols.empty:
            return result  

        X = result.drop(columns=['Sales'])  
        y = result['Sales']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

        categorical_transformer = ColumnTransformer(
            transformers=[
                ('OneHot', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
            ],
            remainder="passthrough"
        )

        X_train_encoded = categorical_transformer.fit_transform(X_train)
        X_test_encoded = categorical_transformer.transform(X_test)

        if isinstance(X_train_encoded, csr_matrix):
            X_train_encoded = X_train_encoded.toarray()
            X_test_encoded = X_test_encoded.toarray()
        
        num_encoded_cols = X_train_encoded.shape[1] - len(categorical_cols)
        column_names = list(categorical_cols) + [f'encoded_{i}' for i in range(num_encoded_cols)]

        X_train_encoded = pd.DataFrame(X_train_encoded, columns=column_names)
        X_test_encoded = pd.DataFrame(X_test_encoded, columns=column_names)
        
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_encoded)
        X_test_scaled = scaler.transform(X_test_encoded)
        
        return X_train_scaled, X_test_scaled, y_train, y_test
    
    return wrapper



In [99]:
def train_classifier(X: np.ndarray, y: np.ndarray, **kwargs: dict) -> keras.Model:
    num_classes = len(np.unique(y))  
    
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    epochs = kwargs.get('epochs', 10)
    batch_size = kwargs.get('batch_size', 32)
    validation_split = kwargs.get('validation_split', 0.2)
    
    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=validation_split)
    
    return model

def predict_sales_by_category(X_test: np.ndarray, model: keras.Model, label_encoder: LabelEncoder) -> pd.DataFrame:
    category_probs = model.predict(X_test)
    predicted_categories = np.argmax(category_probs, axis=1)
    predicted_category_labels = label_encoder.inverse_transform(predicted_categories)
   
    result_df = pd.DataFrame({
        'Sales': X_test[:, 0],
        'Category': predicted_category_labels,
        'Probability': np.max(category_probs, axis=1)
    })

    return result_df


@one_hot_encode_all_categorical
def load_data() -> pd.DataFrame:
    df = pd.DataFrame(dados)
    print(df.columns)
    return df


label_encoder = LabelEncoder()


model = train_classifier(X_train_encoded.values, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)


predictions = predict_sales_by_category(X_test_encoded.values, model, label_encoder)


print(predictions)


